# (Iterative) Qubit Coupled Cluster method with Tequila

Welcome! In this Tequila tutorial, we will review how to use the Qubit Coupled Cluster (QCC) methodology implemented in Tequila, along with some extensions, for targeting the ground state energy of a given electronic Hamiltonian. 

## The QCC formalism in a nutshell 

The QCC methodology is a system-specific approach to constructing unitary ansatze with tailorable circuit depth. The QCC unitary is constructed as a product of 1-parameter Pauli term rotations,

$$
\hat U(\boldsymbol{\tau}) = \prod_{i=1}^{N_{gen}} \exp(-i \tau_i \hat P_i / 2)
$$

and the optimal QCC energy is

$$
\min_{\boldsymbol{\tau}} E(\boldsymbol{\tau}) = \min_{\boldsymbol{\tau}} \left \langle 0 | \hat U^\dagger(\boldsymbol{\tau}) \hat H \hat U(\boldsymbol{\tau}) | 0 \right \rangle,
$$

where $|0 \rangle$ is a computational basis state and $\hat H$ is the $N$-qubit mapped electronic Hamiltonian. In analogy to other system-specific ("_adaptive_") ansatz approaches, the individual Pauli terms $\hat P_i$ are included in the QCC unitary based on an **energy response estimate** (i.e., energetic gradients),

$$
\frac{\partial E}{\partial \tau_i} \Big|_{\boldsymbol{\tau} = 0} \propto \left \langle 0 |  \hat H \hat P | 0 \right \rangle,
$$

where Pauli terms with the highest magnitude of $\frac{\partial E}{\partial \tau_i} |_0$ are included. However, unlike other adaptive approaches, the QCC methodology does not _a priori_ restrict the set of generators ("operator pool") to be considered in ranking by the energy response estimate. The QCC methodology **constructively identifies** highest ranking Pauli terms from the set of all possible $4^N - 1$ candidates by means of an **efficient classical algorithm**, exploiting a naturally arising equivalence class structure amongst the energy gradients.

Below, we show how to run straightforward QCC calculations using Tequila to estimate electronic ground state energies. We then introduce the **iterative** extension to the QCC method, and demonstrate how ground state energy estimates can be systematically improved upon.

### Introductory example: 4e4o H$_2$O model

Here, we show how one can use the `iterativeQCC` class to perform QCC calculations, taking a $4$ electron in $4$ orbitals active space model of H$_2$O in the 6-31G basis set. The resulting qubit-space Hamiltonian, obtained via the Jordan-Wigner transform, is defined over $8$ qubits. While the scope of the `iterativeQCC` can perform _iterative_ QCC calculations (more on that later!), the QCC procedure can be performed as easily as a single method call. We illustrate below.

For the geometry setup, we use a ∠HOH angle of $\theta = 107.6$ degrees, and a symmetric O-H bond length of $R = 1.0$ Angstrom. The Euclidean $(x, y, z)$ nuclear coordinates for the oxygen and two hydrogens are respectively given by:
* $(0, 0, 0)$
* $( -R \sin(\theta / 2), R \cos(\theta / 2) , 0)$
* $( R \sin(\theta / 2), R \cos(\theta / 2), 0 )$


In [1]:
import math
import tequila as tq

R = 1.0
angle = math.radians(107.6 / 2)
x = R * math.sin(angle)
y = R * math.cos(angle)

xyz = ''.join(["O 0.0 0.0 0.0\n",
               "H {}  {}  0.0\n".format(-x, y),
               "H {}  {}  0.0".format(x, y)
              ]
             )

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}

h2o = tq.quantumchemistry.Molecule(geometry=xyz, basis_set = basis, active_orbitals = active)

energy_fci = h2o.compute_energy('fci') #compute the FCI energy for error analysis later
print('FCI energy: {}'.format(energy_fci))

FCI energy: -75.98980141090206


### Qubit Coupled Cluster calculation
Let's initialize an instance of `IterativeQCC`, which takes as argument an instance of `tq.Molecule`, for which the ground state energy will be estimated. In our example, we've assigned our `tq.Molecule` to `h2o`:

In [2]:
from tequila.apps.qcc.qcc import IterativeQCC

iqcc_solver = IterativeQCC(h2o)

We are ready to run a QCC calculation! This is done by calling the `do_qcc` method. We can specify how many generators we include in $\hat U (\boldsymbol{\tau})$ by the `n_gen` argument, which defaults to `n_gen = 1`. For example, let's try `n_gen = 6`.

In [3]:
iqcc_solver.do_qcc(n_gen = 6)


 Gradient partitions:
[1. 1. 0. 0. 0. 0. 1. 1.] : 0.07584370394682932
[1. 0. 0. 1. 0. 1. 1. 0.] : 0.037107211651728214
[0. 1. 1. 0. 1. 0. 0. 1.] : 0.037107211651728214
[0. 0. 1. 1. 1. 1. 0. 0.] : 0.03099377018308308
[1. 1. 0. 0. 1. 1. 0. 0.] : 0.030078815043935028
[0. 0. 1. 1. 0. 0. 1. 1.] : 0.024893722737202174
[1. 0. 1. 0. 1. 0. 1. 0.] : 0.019817842532512578
[0. 1. 0. 1. 0. 1. 0. 1.] : 0.019817842532512578
[1. 0. 0. 1. 1. 0. 0. 1.] : 0.017289369119215636
[0. 1. 1. 0. 0. 1. 1. 0.] : 0.017289369119215636
[1. 0. 0. 0. 0. 0. 1. 0.] : 1.0988597593589167e-06
[0. 1. 0. 0. 0. 0. 0. 1.] : 1.098859759331161e-06

Selected QCC generators:
Y(0)X(1)X(6)X(7)
Y(0)X(3)X(5)X(6)
Y(1)X(2)X(4)X(7)
Y(2)X(3)X(4)X(5)
Y(0)X(1)X(4)X(5)
Y(2)X(3)X(6)X(7)
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : COBYLA
Objective       : 1 expectationvalues

active

E=-75.98839868  angles= {t0: 0.09501698469558141, t1: 0.060819649393046585, t2: 0.056365648757061126, t3: 0.052840653306991114, t4: 0.03788562798567562, t5: 0.0352276328597904}  samples= None
E=-75.98840848  angles= {t0: 0.09491674075682133, t1: 0.060492336083301936, t2: 0.060148466380331535, t3: 0.04725633934483797, t4: 0.04351043939382037, t5: 0.034239676831249094}  samples= None
E=-75.98840681  angles= {t0: 0.09381828882614228, t1: 0.06103369717160959, t2: 0.06054762543963397, t3: 0.046363023099827115, t4: 0.04234951803183564, t5: 0.03413993457970618}  samples= None
E=-75.98840000  angles= {t0: 0.09302168087587867, t1: 0.06159151095418504, t2: 0.06126541458860839, t3: 0.04913609489850341, t4: 0.04584483608741827, t5: 0.03376125280512691}  samples= None
E=-75.98840574  angles= {t0: 0.09395801140918217, t1: 0.06004481430473173, t2: 0.05894500101545094, t3: 0.04684945413740586, t4: 0.04403084648027314, t5: 0.03513998320642437}  samples= None
E=-75.98840500  angles= {t0: 0.0979211370350

We can check various quantities involved in the QCC calculation, such as the energy gradient partitions, the individual generators which were selected, the optimized QCC energy and parameters, and so on. The interested user can check the `iterativeQCC` documentation to see more on the metadata available through class attributes.

In [4]:
print('The generators selected for the QCC ansatz, along with their optimized angles:')
for g,t in zip(iqcc_solver.generators[0], iqcc_solver.angles[0]):
    print('{} : {}, {} = {}'.format(g, iqcc_solver.generators[0][g], t, iqcc_solver.angles[0][t]))

print('\nError (Hartree) from FCI: {}'.format(iqcc_solver.energy - energy_fci))

The generators selected for the QCC ansatz, along with their optimized angles:
g0 : Y(0)X(1)X(6)X(7), t0 = 0.09655772668966027
g1 : Y(0)X(3)X(5)X(6), t1 = 0.06065537576671671
g2 : Y(1)X(2)X(4)X(7), t2 = 0.058809236603816435
g3 : Y(2)X(3)X(4)X(5), t3 = 0.04930054786792925
g4 : Y(0)X(1)X(4)X(5), t4 = 0.04118298389830415
g5 : Y(2)X(3)X(6)X(7), t5 = 0.03382882169472854

Error (Hartree) from FCI: 0.0013904103936397405


## Going beyond one-shot QCC: The iterative QCC approach

For a number of possible reasons, QCC may not produce energy estimates which are within a satisfactory error threshold. For instance, if one is running on quantum hardware, errors due to noise may begin to significantly reduce the accuracy of the energy estimator as $N_{gen}$ is increased. Convergence to a local energy minimum may also be a concern when increasing $N_{gen}$ due to the growing dimensionality of the parameter space. 

Luckily, one can extend the QCC method to an **iterative** framework, which circumvents these pitfalls. The **iterative Qubit Coupled Cluster (iQCC)** approach consists of iterating over the following steps:

1. **Selection**: Selection of QCC generators for the current step Hamiltonian.
2. **Optimization**: Optimization of QCC unitary for the current step Hamiltonian.
3. **Dressing**: Updating the Hamiltonian via the _optimized_ QCC unitary.

Hence, the standard QCC method can be considered as performing Steps 1 and 2 of the first iQCC iteration. A crucial step in the iQCC method is to update the Hamiltonian by means of a **similarity transformation** ("_dressing_") with the current iteration's optimized QCC unitary,

$$
\hat H' = \hat U^\dagger (\boldsymbol{\tau}^{*}) \hat H \hat U(\boldsymbol{\tau}^{*}).
$$

Importantly, the optimized QCC parameters $\boldsymbol{\tau}^{*}$ are _fixed_ when similarity transforming the Hamiltonian, meaning the number of simultaneously optimized parameters does not grow as we iterate this procedure.

Let's use this approach to systematically improve our estimate of the H$_2$O ground state energy! From the above example, we've already seen how to use `iterativeQCC` to perform one-shot QCC energy calculations. We just need to update the qubit-space Hamiltonian with the QCC unitary to perform a full iQCC step. The QCC calculation along with post-processing the Hamiltonian can all be done by a single call to `iterativeQCC`'s `do_iteration` method.

To avoid `iterativeQCC` carrying existing metadata from the previous one-shot QCC calculation, let's reinstantiate it and run a iQCC computation of $3$ iterations, where $6$ generators are included in each step's QCC unitary.

In [5]:
iqcc_solver = IterativeQCC(h2o)

n_iter = 3

for _ in range(n_iter):
    iqcc_solver.do_iteration(n_gen=6)



 Gradient partitions:
[1. 1. 0. 0. 0. 0. 1. 1.] : 0.07584370394682932
[1. 0. 0. 1. 0. 1. 1. 0.] : 0.037107211651728214
[0. 1. 1. 0. 1. 0. 0. 1.] : 0.037107211651728214
[0. 0. 1. 1. 1. 1. 0. 0.] : 0.03099377018308308
[1. 1. 0. 0. 1. 1. 0. 0.] : 0.030078815043935028
[0. 0. 1. 1. 0. 0. 1. 1.] : 0.024893722737202174
[1. 0. 1. 0. 1. 0. 1. 0.] : 0.019817842532512578
[0. 1. 0. 1. 0. 1. 0. 1.] : 0.019817842532512578
[1. 0. 0. 1. 1. 0. 0. 1.] : 0.017289369119215636
[0. 1. 1. 0. 0. 1. 1. 0.] : 0.017289369119215636
[1. 0. 0. 0. 0. 0. 1. 0.] : 1.0988597593589167e-06
[0. 1. 0. 0. 0. 0. 0. 1.] : 1.098859759331161e-06

Selected QCC generators:
Y(0)X(1)X(6)X(7)
Y(0)X(3)X(5)X(6)
Y(1)X(2)X(4)X(7)
Y(2)X(3)X(4)X(5)
Y(0)X(1)X(4)X(5)
Y(2)X(3)X(6)X(7)
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : COBYLA
Objective       : 1 expectationvalues

active

E=-75.98839868  angles= {t0: 0.09501698469558141, t1: 0.060819649393046585, t2: 0.056365648757061126, t3: 0.052840653306991114, t4: 0.03788562798567562, t5: 0.0352276328597904}  samples= None
E=-75.98840848  angles= {t0: 0.09491674075682133, t1: 0.060492336083301936, t2: 0.060148466380331535, t3: 0.04725633934483797, t4: 0.04351043939382037, t5: 0.034239676831249094}  samples= None
E=-75.98840681  angles= {t0: 0.09381828882614228, t1: 0.06103369717160959, t2: 0.06054762543963397, t3: 0.046363023099827115, t4: 0.04234951803183564, t5: 0.03413993457970618}  samples= None
E=-75.98840000  angles= {t0: 0.09302168087587867, t1: 0.06159151095418504, t2: 0.06126541458860839, t3: 0.04913609489850341, t4: 0.04584483608741827, t5: 0.03376125280512691}  samples= None
E=-75.98840574  angles= {t0: 0.09395801140918217, t1: 0.06004481430473173, t2: 0.05894500101545094, t3: 0.04684945413740586, t4: 0.04403084648027314, t5: 0.03513998320642437}  samples= None
E=-75.98840500  angles= {t0: 0.0979211370350

E=-75.98291837  angles= {t0: 0.00999999999999999, t1: 0.01, t2: 0.0984267560139372, t3: -0.0783499225854308, t4: 0.01, t5: 0.01}  samples= None
E=-75.95897268  angles= {t0: 0.00940609858905347, t1: 0.01108899683202912, t2: 0.18251973990837106, t3: 0.18442045676271382, t4: 0.009554425684481784, t5: -0.03809060651704819}  samples= None
E=-75.88793406  angles= {t0: 0.003217832590783869, t1: 0.022436001475659426, t2: 0.002611626711768897, t3: 0.022597476295107173, t4: 0.0049116813912341226, t5: 0.25911879218038614}  samples= None
E=-75.97823836  angles= {t0: -0.026001557560455905, t1: -0.014324943913290264, t2: -0.02678591026831563, t3: -0.015251559671231414, t4: -0.049913782977685145, t5: -0.08031433499750182}  samples= None
E=-75.98281001  angles= {t0: -0.0054828630557865795, t1: 0.005411253059853282, t2: -0.005978375499980438, t3: 0.005128705673001976, t4: -0.013627675543980039, t5: 0.06299216271737589}  samples= None
E=-75.98666163  angles= {t0: 0.010814471410138287, t1: 0.010800300363

E=-75.98938213  angles= {t0: -0.03270739236263313, t1: -0.02862893784971466, t2: -0.029675077407287337, t3: -0.02410080731923802, t4: -0.009647761890853482, t5: 0.0012832025496205633}  samples= None
E=-75.98937388  angles= {t0: -0.03333982244209062, t1: -0.028801793080296272, t2: -0.02645111496103779, t3: -0.023295234091834018, t4: -0.010044277515275132, t5: 0.003188152839368307}  samples= None
E=-75.98937754  angles= {t0: -0.03384868922467511, t1: -0.02909340180552726, t2: -0.03053434960061023, t3: -0.023475068708391934, t4: -0.010559649290613456, t5: 0.0018619735154667165}  samples= None
E=-75.98938077  angles= {t0: -0.03175073416951077, t1: -0.0316185756811501, t2: -0.02939556757993765, t3: -0.0255078180918094, t4: -0.009324329479040725, t5: -0.0005176989518930275}  samples= None
E=-75.98938390  angles= {t0: -0.03308192440453291, t1: -0.028060827280803342, t2: -0.029373090474448228, t3: -0.025432717289433575, t4: -0.010837115722060286, t5: 0.0010150037714942788}  samples= None
E=-75

E=-75.98197708  angles= {t0: -0.09792071538874579, t1: -0.019462820092215533, t2: -0.005934524796104618, t3: -0.04269977357538755, t4: 0.0013109143450833922, t5: 0.008159892040719662}  samples= None
E=-75.98915439  angles= {t0: 0.009944396809898344, t1: 0.010611639437421227, t2: 0.011027105629387642, t3: 0.017771425023064454, t4: 0.014048557405844538, t5: 0.07187109175982262}  samples= None
E=-75.98911727  angles= {t0: 0.024772188343012792, t1: 0.005047917876104883, t2: 0.006650105924426147, t3: -0.015194125560131765, t4: 0.0014503605365421765, t5: 0.013841841509535558}  samples= None
E=-75.98738958  angles= {t0: 0.052218485015331356, t1: -0.013246969734066116, t2: -1.341701036457893e-05, t3: 0.039734654910291364, t4: 0.03323431257239341, t5: 0.0023073790146397877}  samples= None
E=-75.98854975  angles= {t0: 0.023320792864446195, t1: 0.05848529644768169, t2: 0.046215347104989875, t3: 0.005746808988673939, t4: 0.016418910143047874, t5: 0.0073280908127897944}  samples= None
E=-75.9888675

E=-75.98975623  angles= {t0: 0.008005213539168977, t1: 0.017335602124686596, t2: -0.03200448722703431, t3: 0.00507588341289777, t4: -0.016212935551442507, t5: 0.026848957609342878}  samples= None
E=-75.98975410  angles= {t0: 0.00713531517125522, t1: 0.017072110698468736, t2: -0.03155915491161564, t3: 0.006891733412572563, t4: -0.01557522154349081, t5: 0.026248835229594026}  samples= None
E=-75.98975619  angles= {t0: 0.007692863823577442, t1: 0.0173286368152462, t2: -0.03031190495019678, t3: 0.003842316531965365, t4: -0.015390098962334599, t5: 0.027035696315361735}  samples= None
E=-75.98975606  angles= {t0: 0.008343043196654413, t1: 0.017341829384403536, t2: -0.03112060376166611, t3: 0.005010061984169284, t4: -0.016352728412898455, t5: 0.025770372008}  samples= None

Optimized QCC parameters:
 t0 : 0.007936055235392243
t1 : 0.017423361365724893
t2 : -0.03111127787067143
t3 : 0.005354646527305358
t4 : -0.016181269784741152
t5 : 0.02659468371762734


   Normal return from subroutine COBY

In [6]:
print('Energy errors (Hartree) relative to the FCI value for the iQCC iterations:\n')
for idx in range(n_iter):
    print('iter {}: {}'.format(idx, iqcc_solver.energies[idx] - energy_fci))

print('\nNumber of Pauli terms in the iQCC Hamiltonians produced at the dressing step (first entry is for the initial Hamiltonian):')
for idx in range(n_iter + 1):
    print('iter {}: {}'.format(idx-1, iqcc_solver.n_terms[idx]))
    
print('\nGenerators and angles used at each step:\n')
for idx in range(n_iter):
    
    print('iter {}:'.format(idx))
    for g,t in zip(iqcc_solver.generators[idx], iqcc_solver.angles[idx]):
        print('{} : {}, {} = {}'.format(g, iqcc_solver.generators[idx][g], t, iqcc_solver.angles[idx][t]))
    print('-'*40 + '\n')

Energy errors (Hartree) relative to the FCI value for the iQCC iterations:

iter 0: 0.0013904103936397405
iter 1: 0.00041349800252987734
iter 2: 4.493941645478117e-05

Number of Pauli terms in the iQCC Hamiltonians produced at the dressing step (first entry is for the initial Hamiltonian):
iter -1: 185
iter 0: 1241
iter 1: 2575
iter 2: 3191

Generators and angles used at each step:

iter 0:
g0 : Y(0)X(1)X(6)X(7), t0 = 0.09655772668966027
g1 : Y(0)X(3)X(5)X(6), t1 = 0.06065537576671671
g2 : Y(1)X(2)X(4)X(7), t2 = 0.058809236603816435
g3 : Y(2)X(3)X(4)X(5), t3 = 0.04930054786792925
g4 : Y(0)X(1)X(4)X(5), t4 = 0.04118298389830415
g5 : Y(2)X(3)X(6)X(7), t5 = 0.03382882169472854
----------------------------------------

iter 1:
g0 : Y(1)X(3)X(5)X(7), t0 = -0.0314432903051042
g1 : Y(0)X(2)X(4)X(6), t1 = -0.029708720749756855
g2 : Y(0)X(3)X(4)X(7), t2 = -0.028310683210951433
g3 : Y(1)X(2)X(5)X(6), t3 = -0.02821002792253945
g4 : Y(0)X(1)X(3)X(5)X(6)X(7), t4 = -0.0100907061440051
g5 : Y(0)X(1)X

In this introductory tutorial, we saw how one can run **QCC** and **iterative QCC calculations** within Tequila. For the $4$e$4$o active space model of H$_2$O in the 6-31G basis, we used a one-shot QCC calculation to provide a chemically accurate estimate of the ground state energy using only $6$ generators and $6$ free parameters. We then used the iterative QCC method to provide **systematically improved** ground state energy estimates. 